# Kanji Analysis
Author: Addison Rogers


In [ ]:
import numpy as np
import seaborn as sns

In [4]:
import os
import pandas as pd

assignments_df = pd.read_csv('data/assignments.csv')
level_df = pd.read_csv('data/level_progressions.csv')
resets_df = pd.read_csv('data/resets.csv')
reviewstats_df = pd.read_csv('data/review_statistics.csv')
reviews_df = pd.read_csv('data/reviews.csv')

In [17]:
print(pd.to_datetime(resets_df.created_at))
print(resets_df.created_at)

0   2017-06-19 13:04:14.202230+00:00
Name: created_at, dtype: datetime64[ns, UTC]
0    2017-06-19T13:04:14.202230Z
Name: created_at, dtype: object
